In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [3]:
#data from kaggle dataset: "Prediction of music genre"
data1 = pd.read_csv("/Users/anniepang/Downloads/music_genre.csv")
#data from kaggle data set: "Spotify Tracks Dataset"
data2 = pd.read_csv("/Users/anniepang/Downloads/dataset.csv")

In [4]:
data1.head()

,instance_id,artist_name,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,obtained_date,valence,music_genre
0,32894.0,Röyksopp,Röyksopp's Night Out,27.0,0.00468,0.652,-1.0,0.941,0.79200,A#,0.115,-5.201,Minor,0.0748,100.889,4-Apr,0.759,Electronic
1,46652.0,Thievery Corporation,The Shining Path,31.0,0.01270,0.622,218293.0,0.890,0.95000,D,0.124,-7.043,Minor,0.0300,115.00200000000001,4-Apr,0.531,Electronic
2,30097.0,Dillon Francis,Hurricane,28.0,0.00306,0.620,215613.0,0.755,0.01180,G#,0.534,-4.617,Major,0.0345,127.994,4-Apr,0.333,Electronic
3,62177.0,Dubloadz,Nitro,34.0,0.02540,0.774,166875.0,0.700,0.00253,C#,0.157,-4.498,Major,0.2390,128.014,4-Apr,0.270,Electronic
4,24907.0,What So Not,Divide & Conquer,32.0,0.00465,0.638,222369.0,0.587,0.90900,F#,0.157,-6.266,Major,0.0413,145.036,4-Apr,0.323,Electronic


In [5]:
data2.head()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


# Data Cleaning

## Cleaned data 1 "tempo" column

In [6]:
data1['tempo'] = data1['tempo'].replace({'\?': ''}, regex=True)

# Convert the 'Tempo' column to numeric 
data1['tempo'] = pd.to_numeric(data1['tempo'], errors='coerce')

# Now 'Tempo' column contains values without "?"

print(len(data1[data1["tempo"] == "?"]))
print(len(data1))


0
50005


In [7]:
import pandas as pd

# data1 'tempo' column
# Convert 'tempo' column to numeric
data1['tempo'] = pd.to_numeric(data1['tempo'], errors='coerce')

# Calculate the average tempo
average_tempo = data1['tempo'].mean()

# Fill missing values in 'tempo' column with the average tempo
data1['tempo'] = data1['tempo'].fillna(average_tempo)
#data1.isna().sum()



# data2 'tempo' column
# Convert 'tempo' column to numeric
data2['tempo'] = pd.to_numeric(data2['tempo'], errors='coerce')

# Calculate the average tempo
average_tempo = data2['tempo'].mean()

# Fill missing values in 'tempo' column with the average tempo
data2['tempo'] = data2['tempo'].fillna(average_tempo)

In [8]:
data1_cleaned = data1
data2_cleaned = data2

data1_cleaned = data1.dropna()
data1_cleaned

,instance_id,artist_name,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,obtained_date,valence,music_genre
0,32894.0,Röyksopp,Röyksopp's Night Out,27.0,0.00468,0.652,-1.0,0.941,0.79200,A#,0.115,-5.201,Minor,0.0748,100.889,4-Apr,0.759,Electronic
1,46652.0,Thievery Corporation,The Shining Path,31.0,0.01270,0.622,218293.0,0.890,0.95000,D,0.124,-7.043,Minor,0.0300,115.002,4-Apr,0.531,Electronic
2,30097.0,Dillon Francis,Hurricane,28.0,0.00306,0.620,215613.0,0.755,0.01180,G#,0.534,-4.617,Major,0.0345,127.994,4-Apr,0.333,Electronic
3,62177.0,Dubloadz,Nitro,34.0,0.02540,0.774,166875.0,0.700,0.00253,C#,0.157,-4.498,Major,0.2390,128.014,4-Apr,0.270,Electronic
4,24907.0,What So Not,Divide & Conquer,32.0,0.00465,0.638,222369.0,0.587,0.90900,F#,0.157,-6.266,Major,0.0413,145.036,4-Apr,0.323,Electronic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50000,58878.0,BEXEY,GO GETTA,59.0,0.03340,0.913,-1.0,0.574,0.00000,C#,0.119,-7.022,Major,0.2980,98.028,4-Apr,0.330,Hip-Hop
50001,43557.0,Roy Woods,Drama (feat. Drake),72.0,0.15700,0.709,251860.0,0.362,0.00000,B,0.109,-9.814,Major,0.0550,122.043,4-Apr,0.113,Hip-Hop
50002,39767.0,Berner,Lovin' Me (feat. Smiggz),51.0,0.00597,0.693,189483.0,0.763,0.00000,D,0.143,-5.443,Major,0.1460,131.079,4-Apr,0.395,Hip-Hop
50003,57944.0,The-Dream,Shawty Is Da Shit,65.0,0.08310,0.782,262773.0,0.472,0.00000,G,0.106,-5.016,Minor,0.0441,75.886,4-Apr,0.354,Hip-Hop


In [9]:
data2_cleaned = data2.dropna()
data2_cleaned

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,4,acoustic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113995,113995,2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Sleep My Little Boy,21,384999,False,0.172,0.2350,...,-16.393,1,0.0422,0.6400,0.928000,0.0863,0.0339,125.995,5,world-music
113996,113996,1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Water Into Light,22,385000,False,0.174,0.1170,...,-18.318,0,0.0401,0.9940,0.976000,0.1050,0.0350,85.239,4,world-music
113997,113997,6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,Best Of,Miss Perfumado,22,271466,False,0.629,0.3290,...,-10.895,0,0.0420,0.8670,0.000000,0.0839,0.7430,132.378,4,world-music
113998,113998,2e6sXL2bYv4bSz6VTdnfLs,Michael W. Smith,Change Your World,Friends,41,283893,False,0.587,0.5060,...,-10.889,1,0.0297,0.3810,0.000000,0.2700,0.4130,135.960,4,world-music


In [10]:
data1_cleaned.isna().sum()

instance_id         0
artist_name         0
track_name          0
popularity          0
acousticness        0
danceability        0
duration_ms         0
energy              0
instrumentalness    0
key                 0
liveness            0
loudness            0
mode                0
speechiness         0
tempo               0
obtained_date       0
valence             0
music_genre         0
dtype: int64

# One Hot encoding -- converted categorical values to numerical values

In [11]:
from sklearn.preprocessing import OneHotEncoder

# Create an instance of OneHotEncoder
onehot_encoder = OneHotEncoder()

#Data 1


#make key, mode into One Hot encoding
#music_genre into label encoding
onehot =OneHotEncoder(sparse=False)
onehot.fit(data1_cleaned[['key', 'mode']])
onehotencoded = onehot.transform(data1_cleaned[['key', 'mode']])
categorical_columns = [f'{col}_{cat}' for i, col in enumerate(data1_cleaned[['key', 'mode']].columns) for cat in onehot.categories_[i]]
onehotdf = pd.DataFrame(onehotencoded, columns=categorical_columns)
concat_data1 = pd.concat([data1_cleaned, onehotdf], axis=1)
# Drop the original categorical columns
df1_encoded = concat_data1.drop(columns = ['key', 'mode'], axis=1)

df1_encoded_cleaned = df1_encoded.dropna()
df1_encoded_cleaned

/Users/anniepang/anaconda3/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,instance_id,artist_name,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,...,key_C#,key_D,key_D#,key_E,key_F,key_F#,key_G,key_G#,mode_Major,mode_Minor
0,32894.0,Röyksopp,Röyksopp's Night Out,27.0,0.004680,0.652,-1.0,0.941,0.792000,0.115,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,46652.0,Thievery Corporation,The Shining Path,31.0,0.012700,0.622,218293.0,0.890,0.950000,0.124,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,30097.0,Dillon Francis,Hurricane,28.0,0.003060,0.620,215613.0,0.755,0.011800,0.534,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,62177.0,Dubloadz,Nitro,34.0,0.025400,0.774,166875.0,0.700,0.002530,0.157,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,24907.0,What So Not,Divide & Conquer,32.0,0.004650,0.638,222369.0,0.587,0.909000,0.157,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,56911.0,Too $hort,Shake That Monkey,55.0,0.000577,0.937,278707.0,0.695,0.000032,0.295,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
49996,20269.0,Unknown Mortal Orchestra,So Good at Being in Trouble,57.0,0.036300,0.829,230147.0,0.435,0.878000,0.119,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
49997,54580.0,Tee Grizzley,First Day Out,75.0,0.171000,0.587,254694.0,0.711,0.000000,0.125,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
49998,64552.0,Logic,Growing Pains III,55.0,0.474000,0.514,246773.0,0.730,0.000000,0.511,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [12]:
df1_encoded_cleaned.isna().sum()

instance_id         0
artist_name         0
track_name          0
popularity          0
acousticness        0
danceability        0
duration_ms         0
energy              0
instrumentalness    0
liveness            0
loudness            0
speechiness         0
tempo               0
obtained_date       0
valence             0
music_genre         0
key_A               0
key_A#              0
key_B               0
key_C               0
key_C#              0
key_D               0
key_D#              0
key_E               0
key_F               0
key_F#              0
key_G               0
key_G#              0
mode_Major          0
mode_Minor          0
dtype: int64

In [13]:
#Data 2

#make key, mode into One Hot encoding
#music_genre into label encoding
onehot =OneHotEncoder(sparse=False)
onehot.fit(data2_cleaned[['key','mode','explicit' ]])
onehotencoded = onehot.transform(data2_cleaned[['key', 'mode', 'explicit']])
categorical_columns = [f'{col}_{cat}' for i, col in enumerate(data2_cleaned[['key', 'mode', 'explicit']].columns) for cat in onehot.categories_[i]]
onehotdf_2 = pd.DataFrame(onehotencoded, columns=categorical_columns)
concat_data2 = pd.concat([data2_cleaned, onehotdf_2], axis=1)
# Drop the original categorical columns
df2_encoded = concat_data2.drop(columns = ['key', 'mode', 'explicit'], axis=1)

df2_encoded_cleaned = df2_encoded.dropna()
df2_encoded_cleaned

/Users/anniepang/anaconda3/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,danceability,energy,loudness,...,key_6,key_7,key_8,key_9,key_10,key_11,mode_0,mode_1,explicit_False,explicit_True
0,0.0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73.0,230666.0,0.676,0.4610,-6.746,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,1.0,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55.0,149610.0,0.420,0.1660,-17.235,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,2.0,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57.0,210826.0,0.438,0.3590,-9.734,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,3.0,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71.0,201933.0,0.266,0.0596,-18.515,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,4.0,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82.0,198853.0,0.618,0.4430,-9.681,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113994,113994.0,4WbOUe6T0sozC7z5ZJgiAA,Lucas Cervetti,Frecuencias Álmicas en 432hz,"Frecuencia Álmica, Pt. 4",22.0,305454.0,0.331,0.1710,-15.668,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
113995,113995.0,2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Sleep My Little Boy,21.0,384999.0,0.172,0.2350,-16.393,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
113996,113996.0,1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Water Into Light,22.0,385000.0,0.174,0.1170,-18.318,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
113997,113997.0,6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,Best Of,Miss Perfumado,22.0,271466.0,0.629,0.3290,-10.895,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0



# Changing Y into numerical values

In [14]:


labelencoder = LabelEncoder()
# data1['key_num'] = onehot_encoder.fit_transform(data1[['key']]).toarray()
# data1['mode_num'] = onehot_encoder.fit_transform(data1[['mode']]).toarray()
df1_encoded_cleaned['music_genre_num'] = labelencoder.fit_transform(df1_encoded_cleaned['music_genre']) #remain using labelencoder for Y variable
#data1_cleaned = data1_cleaned.dropna(inplace=True)




# data2['explicit_num'] = labelencoder.fit_transform(data2[['explicit']])
df2_encoded_cleaned['track_genre_num'] = labelencoder.fit_transform(df2_encoded_cleaned['track_genre'])
df2_encoded_cleaned

/var/folders/10/vhdqq7rj36x398hvpd822r040000gn/T/ipykernel_76712/14433952.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_encoded_cleaned['music_genre_num'] = labelencoder.fit_transform(df1_encoded_cleaned['music_genre']) #remain using labelencoder for Y variable
/var/folders/10/vhdqq7rj36x398hvpd822r040000gn/T/ipykernel_76712/14433952.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_encoded_cleaned['track_genre_num'] = labelencoder.fit_transform(df2_encoded_cleaned['track_genre'])


,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,danceability,energy,loudness,...,key_7,key_8,key_9,key_10,key_11,mode_0,mode_1,explicit_False,explicit_True,track_genre_num
0,0.0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73.0,230666.0,0.676,0.4610,-6.746,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0
1,1.0,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55.0,149610.0,0.420,0.1660,-17.235,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0
2,2.0,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57.0,210826.0,0.438,0.3590,-9.734,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0
3,3.0,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71.0,201933.0,0.266,0.0596,-18.515,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0
4,4.0,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82.0,198853.0,0.618,0.4430,-9.681,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113994,113994.0,4WbOUe6T0sozC7z5ZJgiAA,Lucas Cervetti,Frecuencias Álmicas en 432hz,"Frecuencia Álmica, Pt. 4",22.0,305454.0,0.331,0.1710,-15.668,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,113
113995,113995.0,2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Sleep My Little Boy,21.0,384999.0,0.172,0.2350,-16.393,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,113
113996,113996.0,1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Water Into Light,22.0,385000.0,0.174,0.1170,-18.318,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,113
113997,113997.0,6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,Best Of,Miss Perfumado,22.0,271466.0,0.629,0.3290,-10.895,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,113


## Dropped missing values and unnecessary categorical columns

In [15]:
df1_encoded_cleaned_dropped = df1_encoded_cleaned.drop(columns=['instance_id', 'artist_name', 'track_name', 'obtained_date', 'music_genre'])
df1_encoded_cleaned_dropped

,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,...,key_D,key_D#,key_E,key_F,key_F#,key_G,key_G#,mode_Major,mode_Minor,music_genre_num
0,27.0,0.004680,0.652,-1.0,0.941,0.792000,0.115,-5.201,0.0748,100.889,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5
1,31.0,0.012700,0.622,218293.0,0.890,0.950000,0.124,-7.043,0.0300,115.002,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5
2,28.0,0.003060,0.620,215613.0,0.755,0.011800,0.534,-4.617,0.0345,127.994,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,5
3,34.0,0.025400,0.774,166875.0,0.700,0.002530,0.157,-4.498,0.2390,128.014,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,5
4,32.0,0.004650,0.638,222369.0,0.587,0.909000,0.157,-6.266,0.0413,145.036,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,55.0,0.000577,0.937,278707.0,0.695,0.000032,0.295,-6.932,0.1340,99.997,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,6
49996,57.0,0.036300,0.829,230147.0,0.435,0.878000,0.119,-10.136,0.0515,103.816,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,6
49997,75.0,0.171000,0.587,254694.0,0.711,0.000000,0.125,-6.330,0.4200,98.008,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,6
49998,55.0,0.474000,0.514,246773.0,0.730,0.000000,0.511,-8.491,0.3870,167.655,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,6


In [16]:
df2_encoded_cleaned_dropped = df2_encoded_cleaned.drop(columns=['Unnamed: 0', 'track_id', 'artists', 'album_name', 'track_name', 'track_genre'])
df2_encoded_cleaned_dropped

,popularity,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,...,key_7,key_8,key_9,key_10,key_11,mode_0,mode_1,explicit_False,explicit_True,track_genre_num
0,73.0,230666.0,0.676,0.4610,-6.746,0.1430,0.0322,0.000001,0.3580,0.7150,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0
1,55.0,149610.0,0.420,0.1660,-17.235,0.0763,0.9240,0.000006,0.1010,0.2670,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0
2,57.0,210826.0,0.438,0.3590,-9.734,0.0557,0.2100,0.000000,0.1170,0.1200,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0
3,71.0,201933.0,0.266,0.0596,-18.515,0.0363,0.9050,0.000071,0.1320,0.1430,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0
4,82.0,198853.0,0.618,0.4430,-9.681,0.0526,0.4690,0.000000,0.0829,0.1670,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113994,22.0,305454.0,0.331,0.1710,-15.668,0.0350,0.9200,0.022900,0.0679,0.3270,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,113
113995,21.0,384999.0,0.172,0.2350,-16.393,0.0422,0.6400,0.928000,0.0863,0.0339,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,113
113996,22.0,385000.0,0.174,0.1170,-18.318,0.0401,0.9940,0.976000,0.1050,0.0350,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,113
113997,22.0,271466.0,0.629,0.3290,-10.895,0.0420,0.8670,0.000000,0.0839,0.7430,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,113


In [17]:
df1_encoded_cleaned_dropped.isna().sum()

popularity          0
acousticness        0
danceability        0
duration_ms         0
energy              0
instrumentalness    0
liveness            0
loudness            0
speechiness         0
tempo               0
valence             0
key_A               0
key_A#              0
key_B               0
key_C               0
key_C#              0
key_D               0
key_D#              0
key_E               0
key_F               0
key_F#              0
key_G               0
key_G#              0
mode_Major          0
mode_Minor          0
music_genre_num     0
dtype: int64

In [18]:
df2_encoded_cleaned_dropped.isna().sum()

popularity          0
duration_ms         0
danceability        0
energy              0
loudness            0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
key_0               0
key_1               0
key_2               0
key_3               0
key_4               0
key_5               0
key_6               0
key_7               0
key_8               0
key_9               0
key_10              0
key_11              0
mode_0              0
mode_1              0
explicit_False      0
explicit_True       0
track_genre_num     0
dtype: int64

In [19]:
# labelencoder = LabelEncoder()
# # data1['key_num'] = onehot_encoder.fit_transform(data1[['key']]).toarray()
# # data1['mode_num'] = onehot_encoder.fit_transform(data1[['mode']]).toarray()
# df1_encoded_cleaned['music_genre_num'] = labelencoder.fit_transform(df1_encoded_cleaned['music_genre']) #remain using labelencoder for Y variable
# #data1_cleaned = data1_cleaned.dropna(inplace=True)




# # data2['explicit_num'] = labelencoder.fit_transform(data2[['explicit']])
# df2_encoded_cleaned['track_genre_num'] = labelencoder.fit_transform(df2_encoded_cleaned['track_genre'])
# df2_encoded_cleaned

# xgboost baseline models

In [72]:
from xgboost import XGBClassifier

In [73]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier

# Splitting Data
X1 = df1_encoded_cleaned_dropped.drop(columns=['music_genre_num'])
y1 = df1_encoded_cleaned_dropped['music_genre_num']
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)

X2 = df2_encoded_cleaned_dropped.drop(columns=['track_genre_num'])
y2 = df2_encoded_cleaned_dropped['track_genre_num']
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=42)

# XGBoost Model Training and Evaluation for Data 1
xgb1 = XGBClassifier()
xgb1.fit(X_train1, y_train1)
y_pred1 = xgb1.predict(X_test1)
accuracy1 = accuracy_score(y_test1, y_pred1)
print("Accuracy for Data 1:", accuracy1)
print("Classification Report for Data 1:")
print(classification_report(y_test1, y_pred1))

# XGBoost Model Training and Evaluation for Data 2
xgb2 = XGBClassifier()
xgb2.fit(X_train2, y_train2)
y_pred2 = xgb2.predict(X_test2)
accuracy2 = accuracy_score(y_test2, y_pred2)
print("Accuracy for Data 2:", accuracy2)
print("Classification Report for Data 2:")
print(classification_report(y_test2, y_pred2))

Accuracy for Data 1: 0.5807580758075808
Classification Report for Data 1:
              precision    recall  f1-score   support

           0       0.46      0.41      0.43      1027
           1       0.83      0.74      0.78      1032
           2       0.63      0.57      0.60      1013
           3       0.85      0.86      0.85       947
           4       0.58      0.60      0.59       995
           5       0.69      0.65      0.67      1007
           6       0.38      0.38      0.38      1005
           7       0.55      0.56      0.55       972
           8       0.37      0.38      0.37       997
           9       0.52      0.69      0.60      1004

    accuracy                           0.58      9999
   macro avg       0.59      0.58      0.58      9999
weighted avg       0.59      0.58      0.58      9999

Accuracy for Data 2: 0.32394736842105265
Classification Report for Data 2:
              precision    recall  f1-score   support

           0       0.26      0.20    

# xgboost improved models

In [75]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid to search
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.01, 0.05]
}

# XGBoost Model Training and Evaluation for Data 1 with hyperparameter tuning
xgb1 = XGBClassifier()
grid_search1 = GridSearchCV(xgb1, param_grid, cv=5, n_jobs=-1)
grid_search1.fit(X_train1, y_train1)
xgb1_best = grid_search1.best_estimator_
y_pred1 = xgb1_best.predict(X_test1)
accuracy1 = accuracy_score(y_test1, y_pred1)
print("Best parameters for Data 1:", grid_search1.best_params_)
print("Accuracy for Data 1:", accuracy1)
print("Classification Report for Data 1:")
print(classification_report(y_test1, y_pred1))

# XGBoost Model Training and Evaluation for Data 2 with hyperparameter tuning
xgb2 = XGBClassifier()
grid_search2 = GridSearchCV(xgb2, param_grid, cv=5, n_jobs=-1)
grid_search2.fit(X_train2, y_train2)
xgb2_best = grid_search2.best_estimator_
y_pred2 = xgb2_best.predict(X_test2)
accuracy2 = accuracy_score(y_test2, y_pred2)
print("Best parameters for Data 2:", grid_search2.best_params_)
print("Accuracy for Data 2:", accuracy2)
print("Classification Report for Data 2:")
print(classification_report(y_test2, y_pred2))


KeyboardInterrupt: 

In [77]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, classification_report
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import GridSearchCV
# from xgboost import XGBClassifier

# # Assuming you have already imported and preprocessed your data

# # Randomly select 20 genres
# selected_genres = np.random.choice(df2_encoded_cleaned_dropped['track_genre_num'].unique(), size=20, replace=False)
# df2_subset = df2_encoded_cleaned_dropped[df2_encoded_cleaned_dropped['track_genre_num'].isin(selected_genres)]

# # Splitting Data for subset
# X2_subset = df2_subset.drop(columns=['track_genre_num'])
# y2_subset = df2_subset['track_genre_num']
# X_train2_subset, X_test2_subset, y_train2_subset, y_test2_subset = train_test_split(X2_subset, y2_subset, test_size=0.2, random_state=42)

# # Encoding target variable
# label_encoder = LabelEncoder()
# y_train2_subset_encoded = label_encoder.fit_transform(y_train2_subset)
# y_test2_subset_encoded = label_encoder.transform(y_test2_subset)

# # Parameter grid for grid search
# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [3, 5, 7],
#     'learning_rate': [0.1, 0.01, 0.001]
# }

# # XGBoost Model Training and Evaluation for Data 2 subset
# xgb2_subset = XGBClassifier()
# grid_search2_subset = GridSearchCV(xgb2_subset, param_grid, cv=5, n_jobs=-1)
# grid_search2_subset.fit(X_train2_subset, y_train2_subset_encoded)
# xgb2_best_subset = grid_search2_subset.best_estimator_
# y_pred2_subset_encoded = xgb2_best_subset.predict(X_test2_subset)

# accuracy2_subset = accuracy_score(y_test2_subset_encoded, y_pred2_subset_encoded)
# print("Accuracy for Data 2 subset:", accuracy2_subset)
# print("Classification Report for Data 2 subset:")
# print(classification_report(y_test2_subset_encoded, y_pred2_subset_encoded))


Accuracy for Data 2 subset: 0.622
Classification Report for Data 2 subset:
              precision    recall  f1-score   support

           0       0.46      0.46      0.46       173
           1       0.51      0.54      0.52       179
           2       0.62      0.62      0.62       226
           3       0.54      0.53      0.53       203
           4       0.55      0.62      0.58       214
           5       0.54      0.61      0.57       199
           6       0.93      0.90      0.92       211
           7       0.67      0.76      0.71       202
           8       0.48      0.41      0.44       208
           9       0.57      0.57      0.57       204
          10       0.65      0.56      0.60       194
          11       0.74      0.70      0.72       203
          12       0.86      0.79      0.83       204
          13       0.45      0.37      0.41       209
          14       0.44      0.43      0.44       194
          15       0.76      0.81      0.79       193
      

In [21]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, MaxPooling1D

# Splitting Data
X1 = df1_encoded_cleaned_dropped.drop(columns=['music_genre_num'])
y1 = df1_encoded_cleaned_dropped['music_genre_num']
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)

X2 = df2_encoded_cleaned_dropped.drop(columns=['track_genre_num'])
y2 = df2_encoded_cleaned_dropped['track_genre_num']
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=42)

# Check the number of unique classes in the training labels for Data 1
num_classes_data1 = len(np.unique(y_train1))

# Check the number of unique classes in the training labels for Data 2
num_classes_data2 = len(np.unique(y_train2))

# Convolutional Neural Network for Data 1
model_cnn1 = Sequential()
model_cnn1.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train1.shape[1], 1)))
model_cnn1.add(MaxPooling1D(pool_size=2))
model_cnn1.add(Flatten())
model_cnn1.add(Dense(10, activation='relu'))
model_cnn1.add(Dense(1, activation='softmax'))
model_cnn1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Reshape input for CNN
X_train1_cnn = X_train1.values.reshape((X_train1.shape[0], X_train1.shape[1], 1))
X_test1_cnn = X_test1.values.reshape((X_test1.shape[0], X_test1.shape[1], 1))

# Train CNN
model_cnn1.fit(X_train1_cnn, y_train1, epochs=10, batch_size=32, verbose=0)
cnn_predictions1 = model_cnn1.predict(X_test1_cnn)

# Convolutional Neural Network for Data 2
model_cnn2 = Sequential()
model_cnn2.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train2.shape[1], 1)))
model_cnn2.add(MaxPooling1D(pool_size=2))
model_cnn2.add(Flatten())
model_cnn2.add(Dense(10, activation='relu'))
model_cnn2.add(Dense(1, activation='softmax'))
model_cnn2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Reshape input for CNN
X_train2_cnn = X_train2.values.reshape((X_train2.shape[0], X_train2.shape[1], 1))
X_test2_cnn = X_test2.values.reshape((X_test2.shape[0], X_test2.shape[1], 1))

# Train CNN
model_cnn2.fit(X_train2_cnn, y_train2, epochs=10, batch_size=32, verbose=0)
cnn_predictions2 = model_cnn2.predict(X_test2_cnn)

# Combine CNN predictions with original features
X_train1_combined = np.concatenate((X_train1, cnn_predictions1), axis=1)
X_test1_combined = np.concatenate((X_test1, model_cnn1.predict(X_test1_cnn)), axis=1)

X_train2_combined = np.concatenate((X_train2, cnn_predictions2), axis=1)
X_test2_combined = np.concatenate((X_test2, model_cnn2.predict(X_test2_cnn)), axis=1)

# XGBoost Model Training and Evaluation for Data 1
xgb1 = XGBClassifier()
xgb1.fit(X_train1_combined, y_train1)
y_pred1 = xgb1.predict(X_test1_combined)
accuracy1 = accuracy_score(y_test1, y_pred1)
print("Accuracy for Data 1 (XGBoost + CNN):", accuracy1)
print("Classification Report for Data 1 (XGBoost + CNN):")
print(classification_report(y_test1, y_pred1))

# XGBoost Model Training and Evaluation for Data 2
xgb2 = XGBClassifier()
xgb2.fit(X_train2_combined, y_train2)
y_pred2 = xgb2.predict(X_test2_combined)
accuracy2 = accuracy_score(y_test2, y_pred2)
print("Accuracy for Data 2 (XGBoost + CNN):", accuracy2)
print("Classification Report for Data 2 (XGBoost + CNN):")
print(classification_report(y_test2, y_pred2))


2024-04-15 17:20:34.580299: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at sparse_xent_op.cc:103 : INVALID_ARGUMENT: Received a label value of 9 which is outside the valid range of [0, 1).  Label values: 1 2 0 1 3 1 5 0 9 6 2 8 0 5 7 3 0 4 2 9 2 1 8 7 9 1 5 4 0 9 3 8


InvalidArgumentError: Graph execution error:

Detected at node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "/Users/anniepang/anaconda3/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/Users/anniepang/anaconda3/lib/python3.10/runpy.py", line 86, in _run_code

  File "/Users/anniepang/anaconda3/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/Users/anniepang/anaconda3/lib/python3.10/site-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/Users/anniepang/anaconda3/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 711, in start

  File "/Users/anniepang/anaconda3/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 199, in start

  File "/Users/anniepang/anaconda3/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/Users/anniepang/anaconda3/lib/python3.10/asynci

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3da032f1-e5ab-4726-ac09-eb3d9c053730' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>